#### Задача 1

Объясняемая переменная y зависит от двух категориальных факторов A и B, причём фактор A имеет 3 уровня, а фактор B - 4 уровня. Результаты наблюдений приведены в таблице:

y = [[2.68, 3.29, 2.88, 4.45],
     [4.12, 4.96, 5.09, 5.22],
     [5.52, 4.50, 5.42, 5.29]]
     
С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную y.

In [1]:
import numpy as np
from scipy import stats

In [2]:
y = np.array([[2.68, 3.29, 2.88, 4.45], 
     [4.12, 4.96, 5.09, 5.22], 
     [5.52, 4.50, 5.42, 5.29]])

Нулевые гипотезы:
$$H_{0a}: \:\: \overline{Y_{1 \ast}} = \dots = \overline{Y_{m \ast}}, \:\:
H_{0b}: \:\: \overline{Y_{\ast 1}} = \dots = \overline{Y_{\ast k}}$$

Для вычисления значений статистик нам вновь понадобятся оценки дисперсий. Они вычисляются похожим образом. Суммы квадратов отклонений:
$$SS_a = k \cdot \displaystyle\sum_{i=1}^m \left( \overline{Y_{i \ast}} - \overline{Y} \right)^2, \:\:
SS_b = m \cdot \displaystyle\sum_{j=1}^k \left( \overline{Y_{\ast j}} - \overline{Y} \right)^2, \:\:
SS_w = \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k \left( y_{ij} - \overline{Y_{i \ast}} - \overline{Y_{\ast j}} + \overline{Y} \right)^2$$

In [3]:
y_mean = y.mean()
yi_mean = y.mean(axis=1)
yj_mean = y.mean(axis=0)

y_mean,yi_mean, yj_mean

(4.451666666666666,
 array([3.325 , 4.8475, 5.1825]),
 array([4.10666667, 4.25      , 4.46333333, 4.98666667]))

In [6]:
m = y.shape[0]
k = y.shape[1]
n = m*k

m, k, n

(3, 4, 12)

In [9]:
SS_a = k * sum((yi_mean - y_mean)**2)
SS_b = m*sum((yj_mean - y_mean)**2)
SS_w = 0
for i in range(m):
    for j in range(k):
        SS_w += (y[i, j] - yi_mean[i]-yj_mean[j]+y_mean)**2

SS_a, SS_b, SS_w

(7.8407166666666654, 1.338166666666669, 1.9298833333333327)

Оценки дисперсий:
$$\sigma_a^2 = \dfrac{SS_a}{m - 1}, \:\: \sigma_b^2 = \dfrac{SS_b}{k - 1}, \:\:
\sigma_w^2 = \dfrac{SS_w}{(k - 1) (m - 1)}$$

In [11]:
sigma2_a = SS_a/(m-1)
sigma2_b = SS_b/(k-1)
sigma2_w = SS_w/((k-1)*(m-1))

sigma2_a, sigma2_b, sigma2_w

(3.9203583333333327, 0.4460555555555563, 0.32164722222222214)

Cтатистика для гипотезы о влиянии фактора $a$:
$$F_a = \dfrac{\sigma_a^2}{\sigma_w^2}$$

В предположении верности гипотезы $H_{0a}$ эта статистика имеет распределение Фишера с параметрами $k_{1a} = m - 1$, $k_{2a} = n - m$, где $n = m \cdot k$ — общее число элементов в выборке. Далее, как обычно, строится критическая область (правосторонняя, поскольку распределение Фишера имеет один хвост), и проводится тест.

Аналогично, для гипотезы о влиянии фактора $b$ статистика:
$$F_b = \dfrac{\sigma_b^2}{\sigma_w^2}$$

Она также имеет распределение Фишера, теперь с параметрами $k_{1b} = k - 1$, $k_{2b} = n - k$.

In [12]:
F_a = sigma2_a/sigma2_w
F_b = sigma2_b/sigma2_w

F_a, F_b

(12.188379262995173, 1.386785038819275)

In [13]:
alpha = 0.05
t_a = stats.t.ppf(1-alpha, m-1, n-m)
t_b = stats.t.ppf(1-alpha, k-1, n-k)

t_a, t_b

(11.919985580355515, 10.353363434801826)

In [14]:
F_a > t_a # Статистика для фактора A попадает в критическую область, значит, нулевая гипотеза отвергается.
# Следовательно влияние фактора A является статистически значимым.

True

In [15]:
F_b > t_b # Статистика для фактора B не попадает в критическую область, значит, нулевая гипотеза не отвергается.
# Следовательно влияние фактора B статистически значимым не является.

False